In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity


In [3]:
d=pd.read_csv("new_data.csv")
d

,customer_id,gender,location_type,longitude,latitude,akeed_order_id,item_count,grand_total,payment_mode,promo_code,...,ven_longitude,vendor_category_en,delivery_charge,serving_distance,is_open,prepration_time,rank,primary_tags,vendor_rating_y,vendor_tag_name
0,0,male,Other,-78.559671,-0.264322,242001.0,1.0,18.7,1,0,...,0.551594,Restaurants,0.0,15.0,1.0,15,1,134,4.3,"American,Burgers,Kids meal,Pasta,Sandwiches,Sm..."
1,0,male,Other,0.449445,-0.264730,242001.0,1.0,18.7,1,0,...,0.551594,Restaurants,0.0,15.0,1.0,15,1,134,4.3,"American,Burgers,Kids meal,Pasta,Sandwiches,Sm..."
2,0,male,Home,0.079665,-1.575867,242001.0,1.0,18.7,1,0,...,0.551594,Restaurants,0.0,15.0,1.0,15,1,134,4.3,"American,Burgers,Kids meal,Pasta,Sandwiches,Sm..."
3,1,male,Other,0.151387,-0.567823,266129.0,1.0,6.0,1,0,...,0.135124,Restaurants,0.7,15.0,1.0,15,1,1065,4.2,"Combos,Fries,Mojitos ,Pasta,Sandwiches"
4,1,male,Home,0.103313,-1.117296,266129.0,1.0,6.0,1,0,...,0.135124,Restaurants,0.7,15.0,1.0,15,1,1065,4.2,"Combos,Fries,Mojitos ,Pasta,Sandwiches"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69548,9178,male,Home,-0.000951,-0.114377,251275.0,2.0,6.0,1,0,...,0.087738,Restaurants,0.7,15.0,1.0,10,1,701,4.3,"Sandwiches,Shawarma,Fresh Juices,Mojitos,Milks..."
69549,9178,male,Home,-0.000951,-0.114377,265699.0,1.0,8.2,1,0,...,0.058017,Sweets & Bakes,0.0,15.0,1.0,13,1,336,4.5,"Cakes,Crepes,Desserts,Donuts,Fresh Juices,Ice ..."
69550,9178,male,Home,-0.000951,-0.114377,285179.0,2.0,4.1,1,0,...,0.091491,Restaurants,0.0,15.0,1.0,10,1,346,4.2,"Breakfast,Free Delivery,Fresh Juices,Fries,Kar..."
69551,9178,male,Home,-0.000951,-0.114377,292159.0,2.0,7.6,1,0,...,-0.062283,Restaurants,0.0,10.0,1.0,15,1,8,4.2,"Burgers,Free Delivery,Milkshakes,Mojitos"


In [4]:
d=d.loc[:,["customer_id","vendor_id","vendor_rating_y",'ven_latitude',"ven_longitude","vendor_tag_name"]]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(d, test_size = 0.30, random_state = 42)

print(X_train.shape)
print(X_test.shape)

(48687, 6)
(20866, 6)


In [5]:
customer_data = X_train.pivot_table(index = 'customer_id', columns = 'vendor_id', values = 'vendor_rating_y').fillna(0)
customer_data.head()

vendor_id,4,13,20,23,28,33,43,44,55,66,...,681,841,843,845,846,849,855,856,858,907
customer_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.2,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,4.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
new_train = X_train.copy()
new_test = X_test.copy()

new_train['vendor_rating_y'] = new_train['vendor_rating_y'].apply(lambda x: 0 if x > 0 else 1)
new_test['vendor_rating_y'] = new_test['vendor_rating_y'].apply(lambda x: 1 if x > 0 else 0)

In [7]:
new_train = new_train.pivot_table(index = 'customer_id', columns = 'vendor_id', values = 'vendor_rating_y').fillna(1)

# The vendor not rated by user is marked as 0 for evaluation 
new_test = new_test.pivot_table(index ='customer_id', columns = 'vendor_id', values = 'vendor_rating_y').fillna(0)

In [8]:
new_train.tail()

vendor_id,4,13,20,23,28,33,43,44,55,66,...,681,841,843,845,846,849,855,856,858,907
customer_id,,,,,,,,,,,,,,,,,,,,,
9173,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9175,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
9176,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9177,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9178,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
new_test.head(2)


vendor_id,4,13,20,23,28,33,43,44,55,66,...,679,681,841,843,845,846,849,855,856,858
customer_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
new_test.head(2)


vendor_id,4,13,20,23,28,33,43,44,55,66,...,679,681,841,843,845,846,849,855,856,858
customer_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# User Similarity Matrix using Cosine similarity as a similarity measure between Users
customer_similarity = cosine_similarity(customer_data)
customer_similarity[np.isnan(customer_similarity)] = 0
print(customer_similarity.shape)

(7772, 7772)


In [12]:
e=pd.DataFrame(customer_similarity)
e.head(5)

,0,1,2,3,4,5,6,7,8,9,...,7762,7763,7764,7765,7766,7767,7768,7769,7770,7771
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.362074,0.0,0.000000,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.206205,0.0,0.0,0.0,0.110498,0.0,0.298099,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.494476,0.0,0.0,0.0,0.276880,0.0,0.000000,0.0


In [201]:
customer_pred_ratings = np.dot(customer_similarity, customer_data)
customer_pred_ratings


array([[ 10.48487302,   4.24409823,   7.2551397 , ...,  10.06877798,
          1.64297274,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   1.68096394,
          1.55801454,   0.        ],
       [213.30469906,  29.18446461,  60.37329474, ...,  11.84701656,
         71.45205522,   0.        ],
       ...,
       [ 21.07663645,   7.70184279,  22.43959963, ...,  40.56597187,
         28.16330076,   0.        ],
       [ 17.22624644,   7.44173095,  13.77348388, ...,   3.20531301,
          8.74168385,   0.        ],
       [  4.13374669,   1.46357323,   2.03034387, ...,   0.83870108,
          0.54450318,   0.        ]])

In [202]:
customer_pred_ratings.shape

(7772, 97)

In [203]:
customer_final_ratings = np.multiply(customer_pred_ratings, new_train)
customer_final_ratings.head()

vendor_id,4,13,20,23,28,33,43,44,55,66,...,681,841,843,845,846,849,855,856,858,907
customer_id,,,,,,,,,,,,,,,,,,,,,
0,10.484873,4.244098,7.255140,3.050056,5.919993,21.923330,8.982040,0.000000,0.000000,3.061225,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.833526,10.068778,1.642973,0.0
1,0.000000,0.000000,0.000000,1.944531,2.500038,0.000000,5.912320,66.533344,13.079811,0.000000,...,4.603383,0.000000,8.139233,1.234094,19.901507,1.295286,0.000000,1.680964,1.558015,0.0
2,0.000000,29.184465,60.373295,35.380784,35.247693,113.853845,26.202826,5.319326,6.240648,3.266193,...,9.029551,0.000000,2.261092,1.248733,5.479847,0.464203,0.000000,11.847017,0.000000,0.0
3,26.631867,15.179845,22.411987,8.949947,7.258053,19.568477,7.948356,1.484585,2.126944,0.000000,...,1.451809,5.915503,1.418813,1.381010,4.073995,0.000000,0.000000,0.000000,5.861660,0.0
4,43.074256,12.322354,35.568654,18.187681,69.372244,0.000000,23.798626,8.443743,6.163845,4.333267,...,2.095020,9.246725,3.339979,0.000000,3.352153,2.354413,0.000000,23.818575,12.873400,0.0


In [205]:
a=customer_final_ratings.iloc[23].sort_values(ascending = False)[0:5]
print(a)


vendor_id
33     54.328724
188    49.516338
298    39.081935
310    37.535072
92     34.217768
Name: 25, dtype: float64


In [206]:
k=customer_similarity[23]
e=pd.DataFrame(k,columns=['Similarity_score'])

g= e.sort_values(by=['Similarity_score'], ascending=False)
g.head(3)

,Similarity_score
7569,1.0
424,1.0
1188,1.0


In [207]:
k=customer_similarity[23]
e=pd.DataFrame(k,columns=['Similarity_score'])

g= e.sort_values(by=['Similarity_score'])
g.head(2)

,Similarity_score
0,0.0
5172,0.0


In [217]:
def vender_recommender(x):
    a=customer_final_ratings.iloc[x].sort_values(ascending = False)[0:1]
    vendor_id=[]
    vendor_rate=[]
    vendor_tag=[]
    vendor_latitude=[]
    vendor_longitude=[]
    for i in a.index:
        vendor_id.append(d[d['vendor_id']==i]['vendor_id'].iloc[0])
        vendor_rate.append(d[d['vendor_id']==i]['vendor_rating_y'].iloc[0])
        vendor_tag.append(d[d['vendor_id']==i]['vendor_tag_name'].iloc[0])
        vendor_latitude.append(d[d['vendor_id']==i]['ven_latitude'].iloc[0])
        vendor_longitude.append(d[d['vendor_id']==i]['ven_longitude'].iloc[0])
        
        B=pd.DataFrame(list(zip(vendor_id, vendor_rate,vendor_tag,vendor_latitude,vendor_longitude)),
               columns =['vendor_ID', 'Rating','Vendor_tag_name','latitude','longitude'])   
        


        
    return B

In [209]:
vender_recommender(23)


,vendor_ID,Rating,Vendor_tag_name,latitude,longitude
0,33,4.6,"Desserts,Mexican",-0.494654,0.743318
1,188,4.6,"Italian,Pasta,Pizzas,Salads,Sandwiches",-0.405660,0.644401
2,298,4.7,"Free Delivery,Fresh Juices,Pastas,Salads,Sandw...",-0.415192,0.643550
3,310,4.8,"Bagels,Desserts,Salads",-0.016829,0.514580
4,92,4.6,"Asian,Fresh Juices,Kids meal",-0.492188,0.744160


In [218]:
A=vender_recommender(23)


In [219]:
B=vender_recommender(424)


In [220]:
C=vender_recommender(5172)


In [221]:
PU= pd.concat([A,B,C])
PU

,vendor_ID,Rating,Vendor_tag_name,latitude,longitude
0,33,4.6,"Desserts,Mexican",-0.494654,0.743318
0,33,4.6,"Desserts,Mexican",-0.494654,0.743318
0,86,4.5,"Cakes,Crepes,Desserts,Donuts,Fresh Juices,Ice ...",-0.498135,0.058017


In [222]:
PU.insert(0,"customer_id" ,["Target_user","Similar_user","Dissimilar_user"],True)


In [223]:
PU.reset_index(drop=True, inplace=True)
PU
#vendor recommendation for target user
#vendor recommendation for similar with target user based on similarity score
#vendor recommendation for dissimilar with target user based on similarity score

,customer_id,vendor_ID,Rating,Vendor_tag_name,latitude,longitude
0,Target_user,33,4.6,"Desserts,Mexican",-0.494654,0.743318
1,similar_user,33,4.6,"Desserts,Mexican",-0.494654,0.743318
2,dissimilar_user,86,4.5,"Cakes,Crepes,Desserts,Donuts,Fresh Juices,Ice ...",-0.498135,0.058017


# Testing on the vendors already rated by the user


In [23]:
test_feature = X_test.pivot_table(index = 'customer_id', columns = 'vendor_id', values = 'vendor_rating_y').fillna(0)
test_similarity = cosine_similarity(test_feature)
test_similarity[np.isnan(test_similarity)] = 0

print(test_similarity)
print("- "*10)
print(test_similarity.shape)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.32582349 0.        ]
 [0.         0.         1.         ... 0.58097668 0.         0.        ]
 ...
 [0.         0.         0.58097668 ... 1.         0.         0.        ]
 [0.         0.32582349 0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
- - - - - - - - - - 
(5731, 5731)


In [24]:
cust_pred_ratings_test = np.dot(test_similarity, test_feature)
cust_pred_ratings_test

array([[  1.62211392,   1.73271259,   0.        , ...,   0.        ,
          0.        ,   1.55801454],
       [171.02948927,  20.28889531,  43.77244818, ...,   0.        ,
          6.69562571,  49.21938595],
       [ 35.38202164,   8.50719236,  20.80537575, ...,   0.        ,
          2.20829908,   1.2271429 ],
       ...,
       [ 33.45153153,  12.29349182,  24.04630309, ...,   0.        ,
          4.80476809,   5.16510908],
       [ 11.14131756,   7.44173095,  13.9209736 , ...,   0.        ,
          6.40988704,  12.8424419 ],
       [  1.49833481,   1.600494  ,   3.28551781, ...,   0.        ,
          2.06376517,   1.36684888]])

In [25]:
final_rating = np.multiply(cust_pred_ratings_test,new_test)
final_rating.head()

vendor_id,4,13,20,23,28,33,43,44,55,66,...,679,681,841,843,845,846,849,855,856,858
customer_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,171.029489,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,44.832651,0.0,0.0,0.0,0.0,0.0,0.0,49.219386
4,0.000000,0.0,0.0,0.0,0.0,500.710928,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [26]:
from sklearn.preprocessing import MinMaxScaler

W = final_rating.copy() 
W = W[W > 0] # only consider non-zero values as 0 means the user haven't rated the movies

minscaler = MinMaxScaler(feature_range = (0.5, 5))
minscaler.fit(W)
predicted = minscaler.transform(W)

print(predicted)

[[       nan        nan        nan ...        nan        nan        nan]
 [0.88765705        nan        nan ...        nan        nan 0.5       ]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]]


In [27]:
total_nonzero = np.count_nonzero(~np.isnan(predicted))
total_nonzero

10090

In [28]:
test_data = X_test.pivot_table(index = 'customer_id', columns = 'vendor_id', values = 'vendor_rating_y')
test_data.head()

vendor_id,4,13,20,23,28,33,43,44,55,66,...,679,681,841,843,845,846,849,855,856,858
customer_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.2
4,NaN,NaN,NaN,NaN,NaN,4.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
matrix = (test_data - predicted)**2
sum_of_squares_err = matrix.sum().sum() # df.sum().sum() by default ignores null values

rmse = np.sqrt(sum_of_squares_err/total_nonzero)
print(rmse)


1.8292172271690994


In [30]:
mae = np.abs(predicted - test_data).sum().sum()/total_nonzero
print(mae)

1.5345004840115655


In [225]:
import pickle

In [226]:
pickle.dump(customer_final_ratings,open("customer_final_ratings.pkl","wb"))